# Static field plots

Depict urban areas as seen by the RCM

comments:
1) Urban fraction and orography have different coordenates for NAM-22 (RegCM4)
2) Make a function to select the urban fraction with xarray based on the orography (delete buffer)

In [1]:
import cartopy.crs as ccrs
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import pandas as pd
import xarray as xr
import numpy as np
from functools import partial
import sys

In [2]:
def plot_static_fields(city, cities, dest, model):
    lonlat = ccrs.PlateCarree() 
    proj = lonlat                  ####
    f = plt.figure(figsize=(12,9))
    ax = plt.axes(projection=proj)
    ords.orog.where(lfds.sftlf > land_thres*100).fillna(cities[city]['vmin']).plot(
    ax=ax, x='lon', y='lat', transform=lonlat, cmap=plt.cm.terrain, 
    vmin=cities[city]['vmin'], vmax=cities[city]['vmax'])
    ufds.sftuf.where(ufds.sftuf > urban_thres).plot(
        ax=ax, x=coords_name(ufds)[0], y=coords_name(ufds)[1], 
        transform=lonlat, cmap=plt.cm.gray_r, vmin=0, vmax=1
    )
    ax.set_extent([clon-size, clon+size, clat-size, clat+size], crs=lonlat)
    plt.title(f'{ords.model_id} (land-frac > {land_thres}, urb-frac > {urban_thres})')
    ax.coastlines(resolution='10m', linewidth=1, color='k')
    t = ucdb_city.plot(ax=ax, transform=lonlat, facecolor="none", edgecolor="red")
    plt.savefig(dest + 'static_fields_' + city.replace(' ', '') 
                + '_' + model + '.png',  bbox_inches='tight')
    plt.close()

In [3]:
def coords_name(ds):
    coords = [c for c in ds.coords]
    if any(np.in1d(coords, 'lon')):
        res = ['lon', 'lat']
    elif any(np.in1d(coords, 'xlon')):
        res = ['xlon', 'xlat']
    return res

In [4]:
cities = {
     'Mexico City' : dict(domain = 'CAM-22', vmin= -1000, vmax = 4000),
     'Buenos Aires' : dict(domain = 'SAM-22', vmin= -400, vmax = 1000),
     'New York' : dict(domain = 'NAM-22', vmin= -400, vmax = 1000),
     'Sydney' : dict(domain = 'AUS-22', vmin= -500, vmax = 2000),
     'Beijing' : dict(domain = 'EAS-22', vmin= -500, vmax = 2000),
     'Tokyo' : dict(domain = 'EAS-22', vmin= -500, vmax = 2000),
     'Jakarta' : dict(domain = 'SEA-22', vmin= -500, vmax = 2000),
     'Johannesburg' : dict(domain = 'AFR-22', vmin= -500, vmax = 4000), # Pretoria center station
     'Riyadh' : dict(domain = 'WAS-22', vmin= -500, vmax = 2000),
     'Berlin' : dict(domain = 'EUR-11', vmin= -400, vmax = 1000),
     'Paris' : dict(domain = 'EUR-11',  vmin= -1000, vmax = 1000),
     'London' : dict(domain = 'EUR-11', vmin= -1000, vmax = 1000),
     'Madrid' : dict(domain = 'EUR-11', vmin= -1000, vmax = 2000),
     'Los Angeles': dict(domain = 'NAM-22', vmin= -1000, vmax = 4000),
     'Montreal': dict(domain = 'NAM-22', vmin= -1000, vmax = 4000),
}

urban_thres = 0.10
land_thres = 0.50
size = 3 # degrees plotted around city center

In [5]:
root = '/lustre/gmeteo/WORK/DATA/CORDEX-FPS-URB-RCC/nextcloud/'
dest = '/home/javi/Research/cordex-fps-urb-rcc/git/CORDEX-CORE-WG/pictures/' # path to save the results
os.makedirs(dest,  exist_ok=True)

In [7]:
ucdb_info = gpd.read_file(root + 'CORDEX-CORE-WG/GHS_FUA_UCD/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg')

## Main Loop

In [28]:
for city in cities:
    # Get the urban center polygon:
    ucdb_city = ucdb_info.query(f'UC_NM_MN =="{city}"').to_crs(crs = 'EPSG:4326')
    # select the city and add the location and domain:
    city_info = pd.read_csv('city_info.csv').query(f'city == "{city}"').to_dict('records')[0]
    # Fix some domains (missing data for the moment in those originally assigned)
    city_info['domain'] = cities[city]['domain']   
    clon = city_info['lon']
    clat = city_info['lat']
    domain = city_info['domain']
    
    #REMO
    ords = xr.open_mfdataset(root  + f'CORDEX-CORE-WG/REMO/orography/orog_{domain}_*REMO*.nc')
    lfds = xr.open_mfdataset(root  + f'CORDEX-CORE-WG/REMO/land-sea-mask/sftlf_{domain}_*REMO*.nc')
    ufds_orig = xr.open_dataset(root + f'CORDEX-CORE-WG/REMO/urbanfraction/orig/{domain}.nc')
    # Fix the raw urban fraction data
    if domain == 'EUR-11':
        ufnp = ufds_orig.var493.values[0,6:-17,6:-5].copy()
    else:
        ufnp = ufds_orig.var493.values[0,9:-9,9:-9].copy()
    ufds = ords.assign(sftuf = (('rlat','rlon'), ufnp, dict(
    long_name = 'Urban fraction',
    standard_name = 'urban_area_fraction',
    units = '1',
    cell_measures = 'area: areacella',
    grid_mapping = 'rotated_latitude_longitude'
    ))).drop_vars('orog')
    # Plot the results
    plot_static_fields(city, cities, dest, 'REMO')

    #RegCM
    ords = xr.open_mfdataset(root  + f'CORDEX-CORE-WG/RegCM/orography/orog_{domain}_*RegCM*.nc')
    lfds = xr.open_mfdataset(root  + f'CORDEX-CORE-WG/RegCM/land-sea-mask/sftlf_{domain}_*RegCM*.nc')
    ufds_orig = xr.open_dataset(root  + f'CORDEX-CORE-WG/RegCM/urbanfraction/orig/{domain}.nc')
    if domain == 'NAM-22':
        ufds_orig = ufds_orig.assign_coords({"xlat": ufds_orig.xlat, "xlon":ufds_orig.xlon})
        #ufds_orig = ufds_orig.assign_coords({"xlat": ords.lat, "xlon":ords.lon})

    # Fix the raw urban fraction data
    ufnp = ufds_orig.urb_2d.values.sum(axis = 0)/100.
    if domain == 'EAS-22':
        xn = 'x'; yn = 'y'
    elif domain == 'NAM-22':
        xn = 'x'; yn = 'y'
    else:
        xn = 'jx'; yn = 'iy'
    ufds = ufds_orig.assign(sftuf = ((yn,xn), ufnp, dict(
    long_name = 'Urban fraction',
    standard_name = 'urban_area_fraction',
    units = '1',
    cell_measures = 'area: areacella',
    grid_mapping = 'crs'
    )))  
    # Plot the results
    plot_static_fields(city, cities, dest, 'RegCM')